In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import  mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import matplotlib.pyplot as plt
import torch
import numpy as np
import torch.nn as nn
from dnn_model import DNN_Model

In [2]:
# Load validation data

validation_data_path = 'data/right_breast_validation_top_20_features_data.csv'
validation_data = pd.read_csv(validation_data_path)

# Split data into predictors and target
X_val = validation_data.iloc[:, :-1].values
y_true = validation_data.iloc[:, -1].values

# Load training data for scaler
training_data_path = 'data/train_top_20_features_data.csv'
training_data = pd.read_csv(training_data_path)

# Split data into predictors (X) and target (y)
X_train = training_data.iloc[:, :-1].values

# Scale features 
scaler = MinMaxScaler()  
X_train = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


In [3]:
# Function to evaluate a single model
def evaluate_model(model_path, X_val_scaled, y_true):
    # Load model
    model = DNN_Model(input_size=X_val.shape[1], hidden_sizes=[32, 16, 8], output_size=1, num_layers=4, dropout=0.1)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    # Convert validation data to torch Tensor
    X_validation_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
    
    # Make predictions
    with torch.no_grad():
        y_pred = model(X_validation_tensor).numpy().flatten()
    
    # Calculating evaluation metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    
    # Calculating percentage differences
    percentage_diff = np.abs((y_true - y_pred) / y_true) * 100
    max_percentage_diff = np.max(percentage_diff)
    min_percentage_diff = np.min(percentage_diff)
    
    return {
        'MAE': mae,
        'MSE': mse,
        'MAPE': mape,
        'Max % Error': max_percentage_diff,
        'Min % Error': min_percentage_diff
    }

# Create empty list to store results
results = []

# Evaluate each fold model
for fold in range(1, 6):
    model_path = f'models/dose_pred_dnns/model_fold_{fold}.pth'
    metrics = evaluate_model(model_path, X_val_scaled, y_true)
    metrics['Fold'] = fold
    results.append(metrics)

# Convert results to DataFrame
df_results = pd.DataFrame(results)

# Rearrange columns to have Fold first
cols = ['Fold'] + [col for col in df_results.columns if col != 'Fold']
df_results = df_results[cols]

# Round values for better display
df_results = df_results.round(2)

# Calculate mean values across all folds 
mean_values = df_results.drop('Fold', axis=1).mean().round(2)
mean_row = pd.DataFrame([{'Fold': 'Mean'} | mean_values.to_dict()])
df_results = pd.concat([df_results, mean_row], ignore_index=True)

# Display the results
title = 'Results - Right Breast (Cross-validation)'
print(title)
print(df_results)

Results - Right Breast (Cross-validation)
   Fold   MAE   MSE  MAPE  Max % Error  Min % Error
0     1  0.20  0.07  4.12        17.90         0.01
1     2  0.20  0.07  4.12        19.37         0.09
2     3  0.19  0.06  3.96        19.02         0.07
3     4  0.19  0.06  3.90        18.05         0.01
4     5  0.19  0.06  3.93        18.11         0.00
5  Mean  0.19  0.06  4.01        18.49         0.04
